In [3]:
install.packages("IntNMF")

Warning message:
“dependency ‘Biobase’ is not available”
also installing the dependencies ‘pkgmaker’, ‘registry’, ‘rngtools’, ‘gridBase’, ‘BiocManager’, ‘NMF’, ‘InterSIM’


Warning message in install.packages("IntNMF"):
“installation of package ‘NMF’ had non-zero exit status”
Warning message in install.packages("IntNMF"):
“installation of package ‘InterSIM’ had non-zero exit status”
Warning message in install.packages("IntNMF"):
“installation of package ‘IntNMF’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
# install.packages("cluster")

In [6]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("Biobase")

install.packages("IntNMF",dependencies=TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'BiocVersion', 'Biobase'

also installing the dependency ‘BiocGenerics’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'cpp11', 'fansi', 'googlesheets4', 'jpeg', 'later', 'RSQLite',
  'xfun'

also installing the dependencies ‘NMF’, ‘InterSIM’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
.libPaths()

[1] "/opt/conda/lib/R/library"

In [9]:
install.packages("tictoc",dependencies=TRUE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [10]:
#############################
#  GA ENSEMBLE CLUSTERING   #
#      22 - FEB - 2021      #
#   Updated: 28 - 04 -2021  #
# Author: Nicolas Camargo   #
#############################

#############
# LIBRARIES #
#############

library("cluster")      # To run PAM and AGNES algorithm
library("fpc")          # To perform validation across different clustering methods
library("mclust")       # To run model-based clustering
library("smacof")       # To run Multidimensional Scaling
library("factoextra")   # For partitioning clustering methods
library("FactoMineR")   # For cluster stats
library("NbClust")      # To determine the optimal number of clusters/graphics
library("dbscan")       # To run density based clustering
library("IntNMF")       # Another NMF package - calculates optimal number of clusters
library("kohonen")      # To run self-organizing map SOMs
library("flashClust")

library("foreach")      # For running in multiple cores (not used)
library("doParallel")   # For running in multiple cores (not used)
library("tidyverse")    # For data manipulation
library("reshape2")     # For data transformation
library("tictoc")
library("dplyr")

In [11]:
select <- dplyr::select     # Solve conflict of packages
hclust <- flashClust::hclust

#############
# FUNCTIONS #
#############


# Function to create bucket lists based on PCA, NMF and HPS
  #' @param bucketList bucket list with recoded raw questions
  #' @param reduceMethodList pcaSol,nmfSol,MDSSol
dimReductBuckets<- function(bucketList,reduceMethodList){
  
  # Set values
  if(reduceMethodList[["pcaSol"]]){    # Euclidean Dist
    pcaSol<- T
  }else{pcaSol<- F}
  if(reduceMethodList[["nmfSol"]]){   # Cosine Dist
    nmfSol<- T
  }else{nmfSol<- F}
  if(reduceMethodList[["MDSSol"]]){   # Correlation Dist
    MDSSol<- T
  }else{MDSSol<- F}
  
  dimBucket<- list()
  count<-0

  if(pcaSol == T){
    count<- count + 1
    pcaList<- lapply(bucketList,function(x) PCA(x, scale.unit = F, ncp = ncol(x),graph = F))
    pcaList<- lapply(pcaList,function(x){
                              dimCon   <- x$eig[,3]                                # Look cumulative variance per
                              ndim     <- length(dimCon)                           # Total number of dimensions
                              gainVar  <- diff(dimCon)                             # Added variance per dim
                              expected_value<- (1/(ndim-1))*100                    # Expected var explained if we had random data (all dim explained the same)
                              dimTake<- length(which(gainVar>=expected_value)) + 1 # Dimensions to take
                              dimTake<- as.data.frame(x$ind$coord[,1:dimTake])     # Coordinates of Dim to take
                              return(dimTake)
                            }
      )
    dimBucket[[count]]<- pcaList
    names(dimBucket)[count]<- "PCADims"
  }
  
  if(nmfSol == T){
    count<- count + 1
    nmfList<- lapply(bucketList,function(x){
                                x<- apply(x,2,function(k) k - min(k)) # Only positive values allowed
                                whichZero<- which(apply(x,2,sd)>0)    # Check which questions are 0 variance
                                x<- x[,whichZero]
                                randomFactor<-matrix(runif(length(x))/10000,byrow = nrow(x),ncol = ncol(x))
                                x2<- x + randomFactor
                                opt.k<- nmf.opt.k(dat = as.matrix(x2), n.runs = 3, n.fold = 2,k.range = 2:8,
                                                  result = TRUE, make.plot = FALSE, maxiter = 50,
                                                  progress = TRUE)
                                bestCluster<- which.max(rowMeans(opt.k)/2*apply(opt.k,1,sd)) + 1 # The one with the highest mean and the lowest SD
                                nmfSol<-tryCatch(nmf.mnnals(as.matrix(x2),bestCluster), error = function(e) nmf.mnnals(as.matrix(x2),bestCluster+1))
                                W_IntNMF<- nmfSol$W
                                W_IntNMF_Scale<- t(apply(W_IntNMF,1,function(x) x/sum(x))) # Scale per row - % participation of resp in cluster
                                return(W_IntNMF_Scale)
                            }) 
    dimBucket[[count]]<- nmfList
    names(dimBucket)[count]<- "NMFDims"
  }
  
  if(MDSSol == T){
    count<- count + 1
    mdsList<- lapply(bucketList,function(x){
                                  x<- apply(x,2,function(k) k - min(k)) # Only positive values allowed
                                  whichZero<- which(apply(x,2,sd)>0)    # Check which questions are 0 variance
                                  x<- x[,whichZero]
                                  randomFactor<-matrix(runif(length(x))/10000,byrow = nrow(x),ncol = ncol(x))
                                  x2<- x + randomFactor
                                  opt.k<- lapply(2:round(2*sqrt(ncol(x2))),function(k) unfolding(as.matrix(x2),k))
                                  bestCluster<- unlist(lapply(opt.k,'[[','stress'))                      # Stress evolution
                                  bestCluster<- abs(diff(bestCluster)/bestCluster[-length(bestCluster)]) # Delta growth
                                  bestCluster<- max(which(bestCluster>=mean(bestCluster))+1)              # When growth flatten
                                  mdsSol<-opt.k[[bestCluster]]                                           # Pick best sol
                                  dimTake<- as.data.frame(mdsSol$conf.row) 
                                  return(dimTake)
                                }) 
    dimBucket[[count]]<- mdsList
    names(dimBucket)[count]<- "MDSDims"
  }
      
      
  return(dimBucket)
  
}


# Calculate frequency of a number per column
  #' @param data: Segment data to count
freqbycol <- function(kdata){
  
  kdata<- as.matrix(kdata)
  if(ncol(kdata)>1){
    uval <- unique(as.vector(kdata))                           # Check unique values
    freq <- t(sapply(uval, function(x) colSums(kdata == x)))   # Count unique values per column
    colnames(freq) <- colnames(kdata)
    rownames(freq) <- uval
  }else{
    freq<- as(table(kdata),"matrix")
  }
  return(freq)
}


# Calculate the cosine similarity between 2 vectors
  #' @param data data set to use to create cosine similarity
cos.sim <- function(data) {
  data<- as.matrix(data) + 1 
  sim <- data / sqrt(rowSums(data * data))
  sim <- sim %*% t(sim)
  cosine<- 1-sim
  cosine<- as.dist(cosine)
  
  return(cosine)
}   


# Run K-means clustering
  #' @param data Data set to apply segmentation
  #' @param segs Number of segments to create
run_kmeans <- function(data, segs){
  cluster <- NULL
  for (i in 1:length(segs)){
   # kmean <- kmeans(data,segs[i], iter.max = 100, nstart = 30) #30 diff runs
    kmean<- tryCatch(kmeans(data,segs[i],iter.max = 100, nstart = 30), error = function(e) kmeans(data,segs[i]-1,iter.max = 100, nstart = 30))
    cluster <- cbind(cluster, kmean$cluster)
  }
  colnames(cluster) <- paste0("seg",segs)
  
  freqkmeans <- freqbycol(cluster)
  
  return(list(k_cluster = cluster, k_freq = freqkmeans))
}


# Run model-based Latent class clustering
  #' @param data Data set to apply segmentation
  #' @param segs Number of segments to create
run_mclust<-function(data,segs){
  
  cluster <- NULL
  for (i in 1:length(segs)){
    LC <- tryCatch(Mclust(data,segs[i]),               # Run mclust algorithm (sum 10 to avoid underflow)
                   error = function(e) Mclust(data+10,segs[i]))  
    cluster <- cbind(cluster,LC$classification)                            # Pick clusters
  }
  
  colnames(cluster) <- paste0("seg",segs)
  freqmclust <- freqbycol(cluster)                      # Calculate frequency per segment
  
  
  return(list(mc_cluster = cluster, mc_freq = freqmclust))
}


# Density based clustering
  #' @param data raw data to run density based clustering
run_dbclust<- function(data){ 
  # Calculate eps
  eps<- kNNdist(data, k = sqrt(nrow(data))/2)                # Calculate distances of the 5 nearest neighbors for each point
  eps<- seq(quantile(eps,0.2),
            quantile(eps,0.9), length.out = 5)  # Create 5 possible distances to run the algorithm
  
  # Run algorithm for multiple eps values
  dbSegments<- lapply(eps, function(x) fpc::dbscan(data, 
                                                   eps = x, 
                                                   MinPts = sqrt(nrow(data))/2, 
                                                   scale = TRUE, method ="raw"))
  
  dbSegments<- sapply(dbSegments,'[[',"cluster")                   # Get all clusters created
  countNoise<- apply(dbSegments,2,function(x) sum(x==0)/length(x)) # check how many points flagged as noise
  dbSegments<- as.matrix(dbSegments[,which(countNoise<0.15)]  )    # Keep only those that flagged 15% of data points as noise
  
  if(ncol(dbSegments)>0){
    colnames(dbSegments)<- paste0("Seg",1:ncol(dbSegments))
    freqdbclust <- freqbycol(dbSegments)                             # Calculate frequency per segment
  }else{
    dbSegments<- rep(1,nrow(data))
    freqdbclust<- NULL
  }
  
  return(list(db_cluster = dbSegments, db_freq = freqdbclust))
}


# Self organizing map algorithm
  #' @param data Data set to apply segmentation
  #' @param segs Number of segments to create
run_som<- function(data,segs){
  # Create the type of grid you want to plot the data - it defines how many nodes per neighborhood
  # Ideally make a neighborhood of at least 10 observations
  sizeNeighbor<- ceiling(sqrt(segs+2))
  #sizeNeighbor<- floor(sqrt(nrow(data)))
  som_grid<- somgrid(xdim = sizeNeighbor, ydim = sizeNeighbor, top = "hexagonal",toroidal = T)
  
  cluster<-NULL
  for(i in 1:length(segs)){
    nSeg<- segs[i]
    # Create the som model
    somModel<- som(as.matrix(data),
                   grid = som_grid, 
                   rlen = nrow(data), 
                   alpha = c(0.05,0.01), 
                   keep.data = TRUE)
    
    # Take codes/neighborhoods created
    somClass<- somModel$unit.classif                       # How each row(observation) was grouped into one of the hexagons
    somCodes<- somModel$codes[[1]]                         # Weights and location of the hexagons
    som_cluster <- cutree(hclust(dist(somCodes)), nSeg)    # Cluster of the hexagons
    segSol<- som_cluster[somClass]                         # Plot observation results
    cluster<- cbind(cluster,segSol)
  }
  
  colnames(cluster)<- paste0("seg",segs)
  
  somfreq<- freqbycol(cluster)
  
  return(list(som_cluster = cluster, som_freq = somfreq))
} 


# Run Diana Clustering
  #' @param dist.mat Distance matrix to apply segmentation
  #' @param segs Vector of number of segments to create
run_diana<-function(dist.mat,segs){
  
  dianas <- diana(dist.mat, diss = TRUE)       # Run pam algorithm
  dianseg <- cutree(as.hclust(dianas), segs)   # Cut into given segments
  colnames(dianseg)<- paste0("Seg",segs)
  freqdianclust <- freqbycol(dianseg)          # Calculate frequency per segment
  
  return(list(dian_clust= dianas, dian_cluster = dianseg, dian_freq = freqdianclust))
}


# Run Pam Clustering
  #' @param dist.mat Distance matrix to apply segmentation
  #' @param segs Vector of number of segments to create
run_pam<-function(dist.mat,segs){
  cluster <- NULL
  for (i in 1:length(segs)){
    pam <- pam(dist.mat,diss=TRUE,segs[i])           # Run pam algorithm
    cluster <- cbind(cluster,pam$clustering)         # Pick clusters
  }
  
  colnames(cluster) <- paste0("seg",segs)
  freqpam <- freqbycol(cluster)                      # Calculate frequency per segment
  
  return(list(pam_cluster = cluster, pam_freq = freqpam))
}


# Run agnes clustering
  #' @param dist.mat Distance matrix to apply segmentation
  #' @param segs Vector of number of segments to create
  #' @param clustmethod Type of clustering method
run_agclust<-function(dist.mat,segs,clustmethod){
  #agclust <- agnes(dist.mat, diss = TRUE, method = clustmethod)      # Run agnes clustering algorithm
  agclust <- hclust(dist.mat, method = clustmethod)
  agseg <- cutree(agclust, segs)                                     # Cut into given segments
  
  freqagclust <- freqbycol(agseg)                                    # Calculate frequency per segment
  
  return(list(ag_clust= agclust, ag_cluster = agseg, ag_freq = freqagclust))
}


# Function to generate distance matrices
  #' @param dataList list of data to create distance matrices for
  #' @param distMethodList euDist, cosDist, corDist
calculateDistances<- function(dataList,distMethodList){
  
  # Check which distance methods to use
  if(distMethodList[["euDist"]]){    # Euclidean Dist
    euDist<- T
  }else{euDist<- F}
  if(distMethodList[["cosDist"]]){   # Cosine Dist
    cosDist<- T
  }else{cosDist<- F}
  if(distMethodList[["corDist"]]){   # Correlation Dist
    corDist<- T
  }else{corDist<- F}
  
  # Euclidean Dist
  if(euDist){
    euclideanDist<- lapply(1:length(dataList),
                           function(x){
                             dataUse<- dataList[[x]]
                             dataUse[is.na(dataUse)]<-0
                             as.matrix(dist(as.matrix(dataUse)))})
  }

  # Cosine Dist
  if(cosDist){
    cosineDist<- lapply(1:length(dataList),
                        function(x){
                          dataUse<- dataList[[x]]
                          dataUse[is.na(dataUse)]<-0
                          as.matrix(cos.sim(dataUse))})
  }

  # Cor Dist
  if(corDist){
    correlationDist<- lapply(1:length(dataList),
                             function(x){
                               corMat<- cor(t(dataList[[x]]),use = "pairwise.complete.obs")
                               corMat[is.na(corMat)]<-0
                               corMat<- 1 - corMat})
  }

  
  distList<- lapply(1:length(dataList),function(x){
                                       dList<- list()
                                        if(euDist)  dList$euDis = euclideanDist[[x]]
                                        if(cosDist) dList$cosDis = cosineDist[[x]]
                                        if(corDist) dList$corDis = correlationDist[[x]]
                                       return(dList)}
                    )
  names(distList)<- names(dataList)
  return(distList)
}


# Function to generate cluster ensembles for both raw data and distance matrix data
  #' @param dataList list of data to create segments for
  #' @param distanceList list of distance matrices to create segments for
  #' @param segs number of segments to generate
  #' @param distMethodList euDist, cosDist, corDist
  #' @param segMethodList mClust,kClust,somClust,agClust,diClust,pamClust
createEnsembles<- function(dataList,distanceList,segs,distMethodList,segMethodList){
  
  # Check which distance methods to use
  if(distMethodList[["euDist"]]){    # Euclidean Dist
    euDist<- T
  }else{euDist<- F}
  if(distMethodList[["cosDist"]]){   # Cosine Dist
    cosDist<- T
  }else{cosDist<- F}
  if(distMethodList[["corDist"]]){   # Correlation Dist
    corDist<- T
  }else{corDist<- F}
  
  # Check which methods to use
  
    # No distance matrix
    if(segMethodList[["mClust"]]){     # Model Based clustering
      mClust<- T
    }else{mClust<- F}
    if(segMethodList[["kClust"]]){     # K-means clustering
      kClust<- T
    }else{kClust<- F}
    if(segMethodList[["somClust"]]){   # Self organizing map clustering
      somClust<- T
    }else{somClust<- F}
  
    # With distance matrix
    if(segMethodList[["agClust"]]){   # Agnes clustering
      agClust<- T
    }else{agClust<- F}
    if(segMethodList[["diClust"]]){   # Diana clustering
      diClust<- T
    }else{diClust<- F}
    if(segMethodList[["pamClust"]]){  # PAM clustering
      pamClust<- T
    }else{pamClust<- F}
  
  # Raw Data
  ensemblesA<- lapply(1:length(dataList),function(x){
                       dataUse<- dataList[[x]]
                       dataUse[is.na(dataUse)]<-0
                       if(mClust){
                         mClust<- run_mclust(dataUse,segs)    # Run Mclust 
                         mClust<- mClust$mc_cluster
                         mClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                       }else{
                         mClustStats<- NULL
                       }
                       if(kClust){
                         kClust<- run_kmeans(dataUse,segs)    # Run Kmeans
                         kClust<- kClust$k_cluster
                         kClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                       }else{
                         kClustStats<- NULL
                       }
                      if(somClust){
                        somClust<- run_som(dataUse,segs)     # Run self organizing map
                        somClust<- somClust$som_cluster
                        somClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                      }else{
                        somClustStats<- NULL
                      }
                      # Run cluster stats to keep best clusters only
                        # KPI_1 = Dunn Index >1 (minimum mean distance between 2 clusters/maximum mean distance within clusters)
                        # KPI_2 = if(silhouette coefficient <0.5,0,1)
                        # KPI_3 = wb ratio <0.5 (average within/average between)
                       globalDist<- dist(dataUse)
                        for(i in 1:(max(segs)-1)){
                          # Mclust
                          mClustS<- tryCatch(cluster.stats(globalDist,mClust[,i]),
                                             error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                          if(is.list(mClustS)){
                             mClustS<- cbind(ifelse(mClustS$dunn2<1,0,1),
                                             prod(ifelse(mClustS$clus.avg.silwidths<0.5,1,0)),
                                             ifelse(mClustS$wb.ratio<0.5,0,1))
                          }
                          if(!is.null(mClustStats)){
                            mClustStats[i,]<- mClustS
                          }
                          
                          # K means
                          kClustS<- tryCatch(cluster.stats(globalDist,kClust[,i]),
                                         error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0)) 
                          if(is.list(kClustS)){
                            kClustS<- cbind(ifelse(kClustS$dunn2<1,0,1),
                                            prod(ifelse(kClustS$clus.avg.silwidths<0.5,1,0)),
                                            ifelse(kClustS$wb.ratio<0.5,0,1))
                          }
                          if(!is.null(kClustStats)){
                            kClustStats[i,]<- kClustS
                          }
                          
                          # SOM
                          somClustS<- tryCatch(cluster.stats(globalDist,somClust[,i]),
                                             error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0)) 
                          if(is.list(somClustS)){
                            somClustS<- cbind(ifelse(somClustS$dunn2<1,0,1),
                                            prod(ifelse(somClustS$clus.avg.silwidths<0.5,1,0)),
                                            ifelse(somClustS$wb.ratio<0.5,0,1))
                          }
                          if(!is.null(somClustStats)){
                            somClustStats[i,]<- somClustS
                          }
                        }
                      
                      # Keep relevant clusters
                      if(!is.null(mClustStats)){   # Mclust
                        keepMClust<-if(length(which(rowSums(mClustStats)>=2))==0) 1:2 else which(rowSums(mClustStats)>=2)
                        mClust<- mClust[,keepMClust]
                      }else{mClust<- NULL}
                       
                      if(!is.null(kClustStats)){   # K means
                        keepKClust<-if(length(which(rowSums(kClustStats)>=2))==0) 1:2 else which(rowSums(kClustStats)>=2)
                        kClust<- kClust[,keepKClust]
                      }else{kClust<-NULL}
                       
                      if(!is.null(somClustStats)){ # SOM
                        keepSomClust<-if(length(which(rowSums(somClustStats)>=2))==0) 1:2 else which(rowSums(somClustStats)>=2)
                        somClust<- somClust[,keepSomClust]
                      }else{somClust<- NULL}
                      
                      # Bind the clusters
                      clusters<- cbind(mClust,kClust,somClust)
                      return(clusters)}
    )

  # Euclidean distance
  euDist2<-euDist
    if(euDist){
      euDist<- lapply(distanceList,"[[","euDis")
      ensemblesB<- lapply(1:length(euDist),function(x){
                          if(agClust){
                            agClust1<- run_agclust(as.dist(euDist[[x]]),segs,clustmethod = "ward")  # Run Agnes
                            agClust1<- agClust1$ag_cluster
                            agClustStats<- matrix(0,nrow =(max(segs)-1),ncol = 3)
                          }else{
                            agClustStats<- NULL
                          }
                          if(diClust){
                            diClust1<- run_diana(as.dist(euDist[[x]]),segs)                         # Run Diana
                            diClust1<- diClust1$dian_cluster
                            diClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            diClustStats<- NULL
                          }
                          if(pamClust){
                            pamClust1<- run_pam(as.dist(euDist[[x]]),segs)                        # Run PAM
                            pamClust1<- pamClust1$pam_cluster
                            pamClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            pamClustStats<- NULL
                          }
                          
                          # Run cluster stats to keep best clusters only
                            # KPI_1 = Dunn Index >1 (minimum mean distance between 2 clusters/maximum mean distance within clusters)
                            # KPI_2 = if(silohete coefficient <0.5,0,1)
                            # KPI_3 = wb ratio <0.5 (average within/average between)
                          for(i in 1:(max(segs)-1)){
                            # Agnes
                            agClustS<- tryCatch(cluster.stats(as.dist(euDist[[x]]),agClust1[,i]),
                                                error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                            if(is.list(agClustS)){
                              agClustS<- cbind(ifelse(agClustS$dunn2<1,0,1),
                                               prod(ifelse(agClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(agClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(agClustStats)){
                              agClustStats[i,]<- agClustS
                            }
                            # Diana
                            diClustS<- tryCatch(cluster.stats(as.dist(euDist[[x]]),diClust1[,i]),
                                                error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                            if(is.list(diClustS)){
                              diClustS<- cbind(ifelse(diClustS$dunn2<1,0,1),
                                               prod(ifelse(diClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(diClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(diClustStats)){
                              diClustStats[i,]<- diClustS
                            }
                            # PAM
                            pamClustS<- tryCatch(cluster.stats(as.dist(euDist[[x]]),pamClust1[,i]),
                                                 error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                            if(is.list(pamClustS)){
                              pamClustS<- cbind(ifelse(pamClustS$dunn2<1,0,1),
                                                prod(ifelse(pamClustS$clus.avg.silwidths<0.5,1,0)),
                                                ifelse(pamClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(pamClustStats)){
                              pamClustStats[i,]<- pamClustS
                            }
                          }
                          
                          # Keep relevant clusters
                          if(!is.null(agClustStats)){
                            keepAgClust<-if(length(which(rowSums(agClustStats)>=2))==0) 1:2 else which(rowSums(agClustStats)>=2)
                            agClust1<- agClust1[,keepAgClust]
                          }else{agClust1<- NULL}
                          if(!is.null(diClustStats)){
                            keepDiClust<-if(length(which(rowSums(diClustStats)>=2))==0) 1:2 else which(rowSums(diClustStats)>=2)
                            diClust1<- diClust1[,keepDiClust]
                          }else{diClust1<- NULL}
                          if(!is.null(pamClustStats)){
                            keepPamClust<-if(length(which(rowSums(pamClustStats)>=2))==0) 1:2 else which(rowSums(pamClustStats)>=2)
                            pamClust1<- pamClust1[,keepPamClust]
                          }else{pamClust1<- NULL}
        
                          # Bind the clusters
                          clusters<- cbind(agClust1,diClust1,pamClust1)
                          return(clusters)
                        }
        )
  }
    
  # Cosine distance
  cosDist2<-cosDist
    if(cosDist){
      cosDist<- lapply(distanceList,"[[","cosDis")
      ensemblesC<- lapply(1:length(cosDist),function(x){
                          if(agClust){
                            agClust1<- run_agclust(as.dist(cosDist[[x]]),segs,clustmethod = "ward")  # Run Agnes
                            agClust1<- agClust1$ag_cluster
                            agClustStats<- matrix(0,nrow =(max(segs)-1),ncol = 3)
                          }else{
                            agClustStats<- NULL
                          }
                          if(diClust){
                            diClust1<- run_diana(as.dist(cosDist[[x]]),segs)                         # Run Diana
                            diClust1<- diClust1$dian_cluster
                            diClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            diClustStats<- NULL
                          }
                          if(pamClust){
                            pamClust1<- run_pam(as.dist(cosDist[[x]]),segs)                        # Run PAM
                            pamClust1<- pamClust1$pam_cluster
                            pamClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            pamClustStats<- NULL
                          }
                          
                          # Run cluster stats to keep best clusters only
                            # KPI_1 = Dunn Index >1 (minimum mean distance between 2 clusters/maximum mean distance within clusters)
                            # KPI_2 = if(silohete coefficient <0.5,0,1)
                            # KPI_3 = wb ratio <0.5 (average within/average between)
                          for(i in 1:(max(segs)-1)){
                            # Agnes
                            agClustS<- tryCatch(cluster.stats(as.dist(cosDist[[x]]),agClust1[,i]),
                                                error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))  
                            if(is.list(agClustS)){
                              agClustS<- cbind(ifelse(agClustS$dunn2<1,0,1),
                                               prod(ifelse(agClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(agClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(agClustStats)){
                              agClustStats[i,]<- agClustS
                            }
                            # Diana
                            diClustS<- tryCatch(cluster.stats(as.dist(cosDist[[x]]),diClust1[,i]),
                                                error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                            if(is.list(diClustS)){
                              diClustS<- cbind(ifelse(diClustS$dunn2<1,0,1),
                                               prod(ifelse(diClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(diClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(diClustStats)){
                              diClustStats[i,]<- diClustS
                            }
                            # PAM
                            pamClustS<-  tryCatch(cluster.stats(as.dist(cosDist[[x]]),pamClust1[,i]),
                                                  error = function(e) list(dunn2 = 0,clus.avg.silwidths = 1, wb.ratio = 0))
                            if(is.list(pamClustS)){
                              pamClustS<- cbind(ifelse(pamClustS$dunn2<1,0,1),
                                                prod(ifelse(pamClustS$clus.avg.silwidths<0.5,1,0)),
                                                ifelse(pamClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(pamClustStats)){
                              pamClustStats[i,]<- pamClustS
                            }
                          }
                          
                          # Keep relevant clusters
                          if(!is.null(agClustStats)){
                            keepAgClust<-if(length(which(rowSums(agClustStats)>=2))==0) 1:2 else which(rowSums(agClustStats)>=2)
                            agClust1<- agClust1[,keepAgClust]
                          }else{agClust1<- NULL}
                          if(!is.null(diClustStats)){
                            keepDiClust<-if(length(which(rowSums(diClustStats)>=2))==0) 1:2 else which(rowSums(diClustStats)>=2)
                            diClust1<- diClust1[,keepDiClust]
                          }else{diClust1<- NULL}
                          if(!is.null(pamClustStats)){
                            keepPamClust<-if(length(which(rowSums(pamClustStats)>=2))==0) 1:2 else which(rowSums(pamClustStats)>=2)
                            pamClust1<- pamClust1[,keepPamClust]
                          }else{pamClust1<- NULL}
                          
                          # Bind the clusters
                          clusters<- cbind(agClust1,diClust1,pamClust1)
                          
                          return(clusters)
                        }
       )
  }
    
  # Correlation distance
  corDist2<-corDist
    if(corDist){
      corDist<- lapply(distanceList,"[[","corDis")
      ensemblesD<- lapply(1:length(corDist),function(x){
                          if(agClust){
                            agClust1<- run_agclust(as.dist(corDist[[x]]),segs,clustmethod = "ward")  # Run Agnes
                            agClust1<- agClust1$ag_cluster
                            agClustStats<- matrix(0,nrow =(max(segs)-1),ncol = 3)
                          }else{
                            agClustStats<- NULL
                          }
                          if(diClust){
                            diClust1<- run_diana(as.dist(corDist[[x]]),segs)                         # Run Diana
                            diClust1<- diClust1$dian_cluster
                            diClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            diClustStats<- NULL
                          }
                          if(pamClust){
                            pamClust1<- run_pam(as.dist(corDist[[x]]),segs)                        # Run PAM
                            pamClust1<- pamClust1$pam_cluster
                            pamClustStats<- matrix(0,nrow =(max(segs)-1),ncol=3)
                          }else{
                            pamClustStats<- NULL
                          }
                          # Run cluster stats to keep best clusters only
                            # KPI_1 = Dunn Index >1 (minimum mean distance between 2 clusters/maximum mean distance within clusters)
                            # KPI_2 = if(silohete coefficient <0.5,0,1)
                            # KPI_3 = wb ratio >0.5 (average within/average between)
                          for(i in 1:(max(segs)-1)){
                            # Agnes
                            agClustS<- tryCatch(cluster.stats(as.dist(corDist[[x]]),agClust1[,i]),
                                                error = function(e) c(0.1,0,1))
                            if(is.list(agClustS)){
                              agClustS<- cbind(ifelse(agClustS$dunn2<1,0,1),
                                               prod(ifelse(agClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(agClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(agClustStats)){
                              agClustStats[i,]<- agClustS
                            }
                            # Diana
                            diClustS<-tryCatch(cluster.stats(as.dist(corDist[[x]]),diClust1[,i]),
                                               error = function(e) c(0.1,0,1))
                            if(is.list(diClustS)){
                              diClustS<- cbind(ifelse(diClustS$dunn2<1,0,1),
                                               prod(ifelse(diClustS$clus.avg.silwidths<0.5,1,0)),
                                               ifelse(diClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(diClustStats)){
                              diClustStats[i,]<- diClustS
                            }
                            # PAM
                            pamClustS<- tryCatch(cluster.stats(as.dist(corDist[[x]]),pamClust1[,i]),
                                                 error = function(e) c(0.1,0,1))
                            if(is.list(pamClustS)){
                              pamClustS<- cbind(ifelse(pamClustS$dunn2<1,0,1),
                                                prod(ifelse(pamClustS$clus.avg.silwidths<0.5,1,0)),
                                                ifelse(pamClustS$wb.ratio<0.5,0,1))
                            }
                            if(!is.null(pamClustStats)){
                              pamClustStats[i,]<- pamClustS
                            }
                          }
                          
                          # Keep relevant clusters
                          if(!is.null(agClustStats)){
                            keepAgClust<-if(length(which(rowSums(agClustStats)>=2))==0) 1:2 else which(rowSums(agClustStats)>=2)
                            agClust1<- agClust1[,keepAgClust]
                          }else{agClust1<- NULL}
                          if(!is.null(diClustStats)){
                            keepDiClust<-if(length(which(rowSums(diClustStats)>=2))==0) 1:2 else which(rowSums(diClustStats)>=2)
                            diClust1<- diClust1[,keepDiClust]
                          }else{diClust1<- NULL}
                          if(!is.null(pamClustStats)){
                            keepPamClust<-if(length(which(rowSums(pamClustStats)>=2))==0) 1:2 else which(rowSums(pamClustStats)>=2)
                            pamClust1<- pamClust1[,keepPamClust]
                          }else{pamClust1<- NULL}
                          
                          # Bind the clusters
                          clusters<- cbind(agClust1,diClust1,pamClust1)
                          
                          return(clusters)
                        }
       )
    }
  
  # Merge ensembles
  totalEnsembles<- lapply(1:length(ensemblesA),function(x){
                            clusters<- ensemblesA[[x]]
                            if(euDist2) clusters<- cbind(clusters,ensemblesB[[x]])
                            if(cosDist2) clusters<- cbind(clusters,ensemblesC[[x]])
                            if(corDist2) clusters<- cbind(clusters,ensemblesD[[x]])
                            return(clusters)
                          })
  
  names(totalEnsembles)<-names(dataList)
  
  return(totalEnsembles)
}


# Function that performs migration between buckets (islands)
  #' @param islandList list where each element is an matrix of ensemble solutions from a bucket (island)
  #' @param migrationRate what % of migration is allowed between buckets (islands)
  #' @param maxMigrant what % of the total bucket (island) population is allowed to enter the bucket (island)
  #' @param maxPop maximum number of ensembles allowed (parents)
migrationIsland<- function(islandList,migrationRate,maxMigrant,maxPop){
  
  nIslands       <- length(islandList)                 # Number of islands (buckets ensemble)
  islandSizes    <- sapply(islandList,ncol)            # Size of the islands (buckets ensemble)
  migrantsIslands<- ceiling(migrationRate*islandSizes) # Number of migrants coming from islands
  migrantsIslands<- lapply(1:length(islandSizes),
                           function(x){
                             randMigrants<- sample(ncol(islandList[[x]]),migrantsIslands[x],replace = F) # Take random migrants numbers
                             randMigrants<- islandList[[x]][,randMigrants]                               # Take actual solution of migrants
                           })
  # Add migrant to the population of the island
  for(i in 1:nIslands){
    migrantPool<- do.call(cbind,migrantsIslands[-i])     # Put all migrants together
    maxMigrantAllowed<- floor(maxMigrant*islandSizes[i]) # Max number of "other" parents allowed in island
    allowedMigrants<- sample(1:ncol(migrantPool),        # Allowed migrants to enter
                             maxMigrantAllowed)
    allowedMigrants<- migrantPool[,allowedMigrants]      # Allowed migrants to enter
    newPopulation<- cbind(islandList[[i]],               # Create new population
                          allowedMigrants)
    # Check that population does not exceed max allowed
    if(ncol(newPopulation)>maxPop){
      randomParents<- sample(1:ncol(newPopulation),      # Take random ensembles
                             maxPop)
      newPopulation<- newPopulation[,randomParents]      # Update new population
    }
    islandList[[i]]<- newPopulation
  }
  
  return(islandList)
  
}

# Function that performs mutation within buckets (islands)
  #' @param islandList list where each element is an matrix of ensemble solutions from a bucket (island)
  #' @param mutationRate what % of segments(labels) is allowed to mutate  (rows, observations)
  #' @param maxMutants what % of the total population of ensembles (island) is allowed to mutate (cols,ensemble sols)
mutationEnsembles<- function(islandList,mutationRate,maxMutants){
  
  nIslands       <- length(islandList)                 # Number of islands (buckets ensemble)
  islandSizes    <- sapply(islandList,ncol)            # Size of the islands (buckets ensemble)
  mutantsIslands <- ceiling(maxMutants*islandSizes)    # Number of mutants per islands (ensembles to adjust)
  islandList     <- lapply(islandList,function(x){     # Rename ensembles names for later match
                                      x<- as.data.frame(x)
                                      names(x)<- paste0("I_",1:ncol(x))
                                      return(x)}
                           )
  mutantsIslands <- lapply(1:length(islandSizes),
                           function(x){
                             randMutants<- sample(ncol(islandList[[x]]),mutantsIslands[x],replace = F) # Take random mutants ensembles (cols)
                             randMutants<- as.matrix(islandList[[x]][,randMutants,drop = FALSE])       # Take actual solution of mutants
                           })
  # Take mutant to the population of the island
  for(i in 1:nIslands){
    mutantPool<- do.call(cbind,mutantsIslands[i])                           # Put all mutants together
    maxMutantsAllowed<- floor(mutationRate*nrow(mutantPool))                # Max number of "other" parents allowed in island
    allowedMutants<- replicate(ncol(mutantPool),sample(1:nrow(mutantPool),  # Allowed mutants per ensemble
                                                       maxMutantsAllowed))
    newMutantLabel<- sapply(1:ncol(mutantPool),function(x){                 # New random label for mutants
                                              round(runif(maxMutantsAllowed,
                                                    min = min(mutantPool[,x]),
                                                    max = max(mutantPool[,x])),0)}
                            )
    for(x in 1:ncol(mutantPool)){
      mutantPool[allowedMutants[,x],x]<- newMutantLabel[,x] # Replace label of mutants
    }
    
    # Replace old ensembles for mutant ensembles
    ensemblesToReplace<- match(colnames(mutantPool),colnames(islandList[[i]]))
    islandList[[i]][,ensemblesToReplace]<- mutantPool
  }
  
  return(islandList)
  
}


# Function that pairs up the parents (ensembles)
  #' @param ensembleMat matrix where each col represent a parent (a possible sol)
  #' @param depthSize number of partners an ensemble may have
parentsPartners<-function(ensembleMat,depthSize){
  
  nParents<- ncol(ensembleMat)          # Number of parents
  depthSize<- min(nParents-1,depthSize) # Check if depth size allowed
  parentList<- list()                   # List to save parents matches
  for(i in 1:nParents){
    partners<- (1:nParents)[-i]             # Potential partners
    partners<- sample(partners,depthSize-1) # Take random candidates
    family<- cbind(ensembleMat[,i],         # Family to generate children
                   ensembleMat[,partners])
    family<- as.list(as.data.frame(family))  # Put them in a list
    parentList[[i]]<- family
    names(parentList)[i]<- paste0("Parent_",i)
  }
  
  return(parentList)
}  


# Function that cross over the ensembles (parents)
  #' @param parentList list with the parents, each element is a possible sol
  #' @param minSize minimum size allowed for the clusters
  #' @param maxSize maximun size allowed for the clusters
crossOver<-function(parentList,minSize,maxSize){
  
  # Put lists in matrices
  parentList<- lapply(parentList,function(x) do.call(cbind,x))
  
  # Determine size range of children
  minSize<- lapply(parentList,function(x) max(min(apply(x,2,max)),minSize))
  maxSize<- lapply(parentList,function(x) min(max(apply(x,2,max)),maxSize))
    # Make sure maxSize >= minSize
    maxSize<- lapply(1:length(maxSize),function(x) max(maxSize[[x]],minSize[[x]]))
  
  # Child size
  childSize<- lapply(1:length(minSize),function(x) round(runif(1,min = minSize[[x]],max = maxSize[[x]]),0))
  
  # One hot encode the parents
  parentsCoded<- lapply(parentList,function(x) apply(x,2,catcode, codetype = 1))
    # Remove non-list elements
    keep<- sapply(parentsCoded,is.list)
    parentsCoded<- parentsCoded[keep]
  parentsCross<- lapply(parentsCoded,function(x) do.call(cbind,x)) # Put them together
  
  # Run SOM clustering
  #somClust<- lapply(parentsCross,function(x) run_som(x,childSize))          # Run SOM algorithm
  #childSol <- somClust$som_cluster # Get child
  
  #mClust<- run_mclust(parentsCross,childSize)
  #childSol<- mClust$mc_cluster
  
  #mClust<- lapply(1:length(parentsCross),function(x) run_mclust(parentsCross[[x]],childSize[[x]])) 
  #childSol<- lapply(mClust,'[[','m_cluster')
  #childSol<- do.call(cbind,childSol)  # Make it a matrix like the original ensembles
  
  #kClust<- lapply(1:length(parentsCross),function(x) run_kmeans(parentsCross[[x]],childSize[[x]])) 
  #childSol<- lapply(kClust,'[[','k_cluster')
  #childSol<- do.call(cbind,childSol)  # Make it a matrix like the original ensembles
  
  hClust<- lapply(1:length(parentsCross),function(x) run_agclust(dist(parentsCross[[x]]),childSize[[x]],clustmethod = "ward")) 
  childSol<- lapply(hClust,'[[','ag_cluster')
  childSol<- do.call(cbind,childSol)  # Make it a matrix like the original ensembles
  
  return(childSol)
}  


# Function that calculates the fitness score of the solution (child)
  #' @param data real data to calculate actual differences
  #' @param dissMat similarity matrix
  #' @param childSol proposed solution (child)
  #' @param simStats fake simulated solution to standarized against
  #' @param weights weights used to calculate final fitness score
fitScore<-function(data,dissMat,childSol,simStats,weights = NULL){
  #' Add weights if not present
  if(is.null(weights)){
    weights<- c(1,0,1,0,1,1,1,1,0,0,0,0,1,0,1,1,0,1)
  }
  
  #' Check methods present - these are should always present in the simulated clustering
  simMethods<- names(simStats) 
  
  #' Calculate KPIs of fitness score for child solution
  fitKPIs<- c("avewithin","mnnd","cvnnd","maxdiameter","widestgap",   
              "sindex", "minsep","asw","dindex","denscut","highdgap",
              "pearsongamma","withinss","entropy","pamc","boot","dmode") 
  
  fitValues<- cqcluster.stats(d = dissMat,
                              clustering = childSol)
  
  #' Standarize solution per segment
  segs<- as.numeric(max(childSol))  # Find out number of segments to check
  methodSolution<- matrix(0,nrow = 1,ncol = length(fitKPIs) + 1)
  colnames(methodSolution)<- c(fitKPIs,"size")
  rownames(methodSolution)<- segs
  methodSolution<- as.data.frame(methodSolution)
  
  simStats<-Reduce(c,simStats)
  seg<- which(names(simStats) == segs)        # Seg to pick
  segValue<- segs                             # Seg Value
    
  allSimStats<-simStats[seg]                  # Take only values for seg N
  simStatSeg<- do.call(rbind,allSimStats)     # Concatenate fit values of all sim methods
  solStatSeg <- fitValues                     # Get segment stats
  
    #' Add size to the KPIs
    solStatSegSize<- as(table(childSol),'matrix')              # Segment Size
    solStatSegSize<- min(solStatSegSize)/sum(solStatSegSize)   # Minimum segment size
    solStatSegSize<- ifelse(solStatSegSize<=0.1,-99,1)
  
  segSolution<-matrix(0,nrow = 1,ncol = length(fitKPIs)+1)
  colnames(segSolution)<- c(fitKPIs,"size")
  segSolution<- as.data.frame(segSolution)
  for (kpi in fitKPIs){
    if(kpi == "dmode"){
      simDmode<- 0.75*simStatSeg[,"dindex"] + 0.25*simStatSeg[,"highdgap"]
      solDmode<- 0.75*segSolution[,"dindex"] + 0.25*segSolution[,"highdgap"]
      standardKPI<- sdstan(solDmode,   # Standardize KPI
                           simDmode)
      segSolution[,kpi]<- standardKPI  # Add to segment solution
    }else{
      #' Check if KPI present in both solution
      simPresent<- kpi %in% names(simStatSeg)
      solPresent<- kpi %in% names(solStatSeg)
      if(simPresent && solPresent){
        simKPI<- simStatSeg[,kpi]     # Value for simulated KPI
        solKPI<- solStatSeg[[kpi]]    # Value for solution KPI
        standardKPI<- sdstan(solKPI,  # Standardize KPI
                             simKPI)
      }else{
        standardKPI<- 0
      }
      segSolution[,kpi]<- standardKPI  # Add to segment solution
    }
      segSolution[,ncol(segSolution)]<- solStatSegSize
  }
  methodSolution[1,]<- segSolution # Add to final Matrix
  
methodSolution[is.na(methodSolution)]<-0
aggScore<- as.matrix(methodSolution) %*% as.matrix(weights)
  
#' Global differences across all variables 
globalMeans<- colMeans(data,na.rm=T) # Global Mean
globalSD<- apply(data,2,sd)          # Global SD
aggResults<-aggregate(x = data, by = list(childSol),mean,na.rm=T)                # Results per cluster
globalSD[globalSD==0]<-10
globalSD[is.na(globalSD)]<-10
globalDiff<- (aggResults[,-1] - globalMeans)/globalSD                                # Standardized results
globalDiff<- mean(colSums(abs(globalDiff),na.rm = T))/(nrow(fullData)*max(childSol)) # Total Score
  
#' Final score
fitScore<- globalDiff + aggScore 
  
  return(fitScore)
}

# Function to fit segment solutions depending of diferent distance matrices or data set
  #' @param data real data to calculate actual differences
  #' @param dissMat similarity matrix
  #' @param children matrix of proposed solutions (children)
  #' @param simStats fake simulated solution to standarized against
  #' @param weights weights used to calculate final fitness score
fitChildren<- function(data,dissMat,children,simStats,weights = NULL,cl){
  fitChild<- parLapply(cl,1:ncol(children),                             # Fitness for each child
                       function(x) fitScore(data =  data, 
                                            dissMat = dissMat,
                                            childSol = children[,x],
                                            simStats = simStats)) 
  
  fitChild<- do.call(rbind,fitChild)                                     # Make them a matrix
  
  return(fitChild)
}


# Function that runs the GA algorithm
  #' @param dataList list with the real data per bucket (island)
  #' @param islandList list where each element is an matrix of ensemble solutions from a bucket (island)
  #' @param simStats fake simulated solution to standarized against
  #' @param optionList list with specs to use
GAensemble<- function(dataList,islandList,simStats,optionList){
  
  #' Distance matrices to use
  fullData<- do.call(cbind,dataList)                     # All data together
  distMatFull<- Dist(fullData)                           # Distance matrix of full data
  distMatIslands<- lapply(dataList,function(x) Dist(x) ) # Distance matrix of islands(buckets) data
  
  # Options in the algorithm to use
    #' [deathRate]     * % of possibilities that died each generation*
    #' [migrationRate] * what % of migration is allowed between buckets (islands) *
    #' [maxMigrant]    * what % of the total bucket (island) population is allowed to enter the bucket (island) *
    #' [mutationRate]  * what % of segments(labels) is allowed to mutate  (rows, observations) *
    #' [maxMutants]    * what % of the total population of ensembles (island) is allowed to mutate (cols,ensemble sols) *
    #' [maxPop]        * maximum number of ensembles allowed (parents) *
    #' [depthSize]     * number of partners an ensemble may have *
    #' [minSize]       * minimum size allowed for the clusters *
    #' [maxSize]       * maximun size allowed for the clusters *
    #' [generations]   * how many generations (epochs) to go *
    #' [traceP]        * to print or not the objective function per generation *
  
  deathRate    <- optionsList[['deathRate']] 
  migrationRate<- optionsList[['migrationRate']] 
  maxMigrant   <- optionsList[['maxMigrant']] 
  mutationRate <- optionsList[['mutationRate']] 
  maxMutants   <- optionsList[['maxMutants']] 
  maxPop       <- optionsList[['maxPop']] 
  depthSize    <- optionsList[['depthSize']] 
  minSize      <- optionsList[['minSize']] 
  maxSize      <- optionsList[['maxSize']]
  generations  <- optionsList[['generations']]
  traceP       <- optionsList[['traceP']]
  weights      <- optionsList[['weights']]
  # Start generations
  iter<- 0
  fitnessVal<- matrix(NA,nrow = length(islandList),ncol = generations+1)  # Matrix to save GEN KPIs and plot
  colnames(fitnessVal)<- paste0("G",0:generations)
  rownames(fitnessVal)<- paste0("Bucket_",1:nrow(fitnessVal))
  bestFit<- list()   # List to save the updated best KPIs value per generation(later on to be used to "kill" solutions)
    for(i in 1:length(islandList)){
      bestFit[[i]]<- 0
    }
  bestList<- list()  # List to save the best solution found so far per bucket
    for(i in 1:length(islandList)){
      bestSol<- runif(nrow(fullData))  # Generate random Data
      bestVal<- 0                      # Initial best value
      bestList[[i]]<- list(bestSol = bestSol, bestVal = bestVal) # save them as list
    }
  convergence<- FALSE
  while((iter<=generations) * (convergence == F)){
    
    # Set parallel computing
    nCores<- detectCores() - 1
    cl<- makeCluster(nCores)
    makeCluster(cl)
    clusterEvalQ(cl,library("cluster"))
    clusterEvalQ(cl,library("fpc"))
    clusterEvalQ(cl,library("clValid"))
    clusterEvalQ(cl,library("flashClust"))
    clusterExport(cl,c("crossOver","catcode","run_agclust","freqbycol","minSize","maxSize","fitScore",
                       "distMatFull","distMatIslands","fullData","simStats","sdstan","dataList"), envir= environment())
    
    ## STEP 1:  Children are created on each island
    parentList<- lapply(islandList,parentsPartners,depthSize)  # Pair-up parents
    tic()
    childrenList<- parLapply(cl,parentList,         # For each pair generate a child
                                function(x){crossOver(parentList =  x,
                                                      minSize = minSize,
                                                      maxSize = maxSize)})  
    toc()
    
    ##' [STEP 2: Children are evaluated on fitness]
      #' * Fitness on full data *
      tic()
      fullFitValsChild<- lapply(1:length(childrenList),function(x){
                                                      fitChildren(data = fullData,
                                                                  dissMat = distMatFull,
                                                                  children = childrenList[[x]],
                                                                  simStats = simStats[[1]],
                                                                  cl = cl)})
      toc()
      
      #' * Fitness on bucket (island) data *
      tic()
      islandFitValsChild<- lapply(1:length(childrenList),function(x){
                                                       fitChildren(data = dataList[[x]],
                                                                  dissMat = distMatIslands[[x]],
                                                                  children = childrenList[[x]],
                                                                  simStats = simStats[[x+1]],
                                                                  cl = cl)})
      toc()
    
    
    ##' [STEP 3: Parents are evaluated on fitness]
      #' * Fitness on full data *
      tic()
      fullFitValsParent<- lapply(1:length(childrenList),function(x){
                                                        fitChildren(data = fullData,
                                                                    dissMat = distMatFull,
                                                                    children = islandList[[x]],
                                                                    simStats = simStats,
                                                                    cl = cl)})
      toc()
      
      #' * Fitness on bucket (island) data *
      tic()
      islandFitValsParent<- lapply(1:length(childrenList),function(x){
                                                          fitChildren(data = dataList[[x]],
                                                                      dissMat = distMatIslands[[x]],
                                                                      children = islandList[[x]],
                                                                      simStats = simStats,
                                                                      cl = cl)})
      toc()
  
      
    ##' [STEP 4: Ensembles to survive next generation]
    tic()
    fitnessScores<- list()
    for(i in 1:length(islandList)){
      # Total population
      parent<- islandList[[i]]  # Ensembles parents
      child<- childrenList[[i]] # Ensembles children
      fullPopulation<- cbind(parent,child)
      nPopKeep<- min(maxPop,ceiling((1-deathRate)*ncol(fullPopulation)))  # Max Number of ensembles to keep
      
      # Parent fit
      fitValsFullParent<- fullFitValsParent[[i]]                    # Full data fit parent
      fitValsIsldParent<- islandFitValsParent[[i]]                  # Island data fit score parent
      finalScoreParent<- (fitValsFullParent + fitValsIsldParent)/2  # Final fit score parent
      
      # Children fit
      fitValsFullChild<- fullFitValsChild[[i]]                      # Full data fit score child
      fitValsIsldChild<- islandFitValsChild[[i]]                    # Island data fit score child
      finalScoreChild<- (fitValsFullChild + fitValsIsldChild)/2     # Final fit score child
      
      finalTotalScores<- c(finalScoreParent,finalScoreChild)        # Bind both scores in parent+child order
      
      popKeep<-order(finalTotalScores,decreasing = T)               # Order from high to low
      popKeep<- popKeep[1:nPopKeep]                                 # Keep allowed population to survive
      newPopulation<- fullPopulation[,popKeep]                      # Population to survive
      
      islandList[[i]]     <- newPopulation                          # Update island list
      fitnessScores[[i]]  <- finalTotalScores[popKeep]              # Save best fitness scores in the same order
      fitnessVal[i,iter+1]<- mean(finalTotalScores[popKeep])        # Update fitness score of generation
    }
    
    islandListFinal<- islandList  # Save unchanged island list to export (before mutations and migrations)
    
    toc()
    
    
    ##' [STEP 5: Migration from other islands (buckets)]
    tic()
    if(migrationRate>0){
      islandList<- migrationIsland(islandList = islandList,
                                   migrationRate = migrationRate,
                                   maxMigrant = maxMigrant,
                                   maxPop = maxPop)
    }
    toc()
    
    
    ##' [STEP 6: Mutation happens]
    tic()
    if(mutationRate>0 && (iter < floor(iter*0.4))){
      islandList<-mutationEnsembles(islandList = islandList,
                                    mutationRate = mutationRate,
                                    maxMutants = maxMutants)
    }
    toc()
    
    
    ##' [STEP 7: Plot trace of fit Vals]
      # Aggregate fitness (use the raw values)
      # Merge all ensemble fitness scores but keep them per island(bucket)
      fitAll     <- mapply(rbind,
                           fullFitValsChild,fullFitValsParent,islandFitValsChild,islandFitValsParent,
                           SIMPLIFY = F) 
      
    meanFit<- colMeans(do.call(rbind,fitAll),na.rm = T)  # Aggregate mean of fitness per KPI
    if(traceP){
      fitnessVal2<- as.matrix(fitnessVal[,which(!is.na(colSums(fitnessVal)))])         # Check which Gens are filled
      if(ncol(fitnessVal2)==1){
        fitnessVal2<- as.matrix(t(fitnessVal2))
      }
      colnames(fitnessVal2)<- colnames(fitnessVal)[which(!is.na(colSums(fitnessVal)))] # Only take those Gens with data
      fitnessVal2<-  as.data.frame(melt(fitnessVal2))                                  # Reshape for GGplot
      print(fitnessVal2 %>% ggplot(aes(x=Var2,y=value, group = Var1, color = Var1)) +
              geom_line() +
              geom_point() + 
              facet_wrap(~Var1,scales = "free")+
              theme(legend.position = "none",
                    axis.title.y = element_blank(),
                    axis.title.x = element_blank()))
    }
    cat("Iter",iter,":", paste("Fit Val=",round(meanFit,2),collapse = " - "), "\n")  # Print mean values
    iter <- iter+1
    
    
    ##' [STEP 8: Update generations]
    # Update the list of best fit per island bucket
    if(iter>1){
      bestFit<- lapply(1:length(bestFit), function(x) colMeans(rbind(apply(fitAll[[x]],2,quantile,0.8,na.rm = T),bestFit[[x]]),na.rm = T))
    }else{
      bestFit<- lapply(1:length(bestFit), function(x) colMeans(rbind(apply(fitAll[[x]],2,quantile,0.8,na.rm = T)),na.rm=T))
    }
      tic()
      # Save best clustering so far (use islandListFinal which is prior mutations or migration)
      for(i in 1:length(islandListFinal)){
        islandPop  <- islandListFinal[[i]]   # Current solutions
        fitBestVals<- fitnessScores[[i]]     # Get fitness scores from current solution
        bestVal    <- which.max(fitBestVals)     # Check which location is the max
        fitBest    <- islandPop[,bestVal]    # Best Solution so far
        bestVal    <- fitBestVals[bestVal]   # Best Value so far
        
        # Update best solution so far
        if(bestList[[i]]$bestVal < bestVal){
          bestList[[i]]$bestSol<- fitBest
          bestList[[i]]$bestVal<- bestVal
        }
      }
      toc()
      stopCluster(cl)
      
  ##' [STEP 9: Check convergence - check if all buckets values are unchanged for 10 periods]
    if(iter>11){
      convergenceCheck<- NULL
      for(i in 1:nrow(fitnessVal)){
        convergenceCheck<- c(convergenceCheck,sd(fitnessVal[i,(iter-10):iter]))   
      }
      if(sum(convergenceCheck)==0){ # If all buckets are not changing in values then stop
        convergence<- TRUE
      }
    }
  }
  
 ##' [STEP 10: Return only unique clusters solutions + best solution found so far]
  islandList<- lapply(islandListFinal, function(x) as.data.frame(x))
  islandList<- lapply(islandList,function(x){
                                  names(x)<- 1:ncol(x)
                                 return(x)}
                      )
  islandList<- lapply(islandList,function(x) t(distinct(as.data.frame(t(x))))) # Removed repeated solutions
  
  return(list(GAEnsembles = islandList, bestSols = bestList))
}


############################################
######      END OF FUNCTIONS     ###########
############################################


In [15]:
rownames(installed.packages())

[1] "abind"         "acepack"       "askpass"       "assertthat"   
  [5] "backports"     "base"          "base64enc"     "Biobase"      
  [9] "BiocGenerics"  "BiocManager"   "BiocVersion"   "bit"          
 [13] "bit64"         "blob"          "boot"          "brew"         
 [17] "brio"          "broom"         "bslib"         "cachem"       
 [21] "callr"         "candisc"       "car"           "carData"      
 [25] "caret"         "cellranger"    "checkmate"     "class"        
 [29] "cli"           "clipr"         "cluster"       "codetools"    
 [33] "colorspace"    "commonmark"    "compiler"      "conflicted"   
 [37] "conquer"       "corrplot"      "covr"          "cowplot"      
 [41] "cpp11"         "crayon"        "credentials"   "crosstalk"    
 [45] "curl"          "data.table"    "datasets"      "DBI"          
 [49] "dbplyr"        "dbscan"        "dendextend"    "DEoptimR"     
 [53] "desc"          "devtools"      "dials"         "DiceDesign"   
 [57] "diffobj"       "digest"        "diptest"       "doParallel"   
 [61] "dplyr"         "DT"            "dtplyr"        "e1071"        
 [65] "ellipse"       "ellipsis"      "evaluate"      "factoextra"   
 [69] "FactoMineR"    "fansi"         "farver"        "fastmap"      
 [73] "flashClust"    "flexmix"       "forcats"       "foreach"      
 [77] "forecast"      "foreign"       "Formula"       "fpc"          
 [81] "fracdiff"      "fs"            "furrr"         "future"       
 [85] "future.apply"  "gargle"        "gdata"         "generics"     
 [89] "gert"          "ggplot2"       "ggpubr"        "ggrepel"      
 [93] "ggsci"         "ggsignif"      "gh"            "git2r"        
 [97] "gitcreds"      "globals"       "glue"          "googledrive"  
[101] "googlesheets4" "gower"         "GPfit"         "graphics"     
[105] "grDevices"     "grid"          "gridBase"      "gridExtra"    
[109] "gtable"        "gtools"        "hardhat"       "haven"        
[113] "heplots"       "here"          "hexbin"        "highr"        
[117] "Hmisc"         "hms"           "htmlTable"     "htmltools"    
[121] "htmlwidgets"   "httpuv"        "httr"          "ids"          
[125] "infer"         "ini"           "InterSIM"      "IntNMF"       
[129] "ipred"         "IRdisplay"     "IRkernel"      "isoband"      
[133] "iterators"     "jomo"          "jpeg"          "jquerylib"    
[137] "jsonlite"      "kernlab"       "KernSmooth"    "knitr"        
[141] "kohonen"       "labeling"      "later"         "lattice"      
[145] "latticeExtra"  "lava"          "lazyeval"      "leaps"        
[149] "lhs"           "lifecycle"     "listenv"       "lme4"         
[153] "lmtest"        "lubridate"     "magrittr"      "maptools"     
[157] "markdown"      "MASS"          "Matrix"        "MatrixModels" 
[161] "matrixStats"   "mclust"        "memoise"       "methods"      
[165] "mgcv"          "mice"          "mime"          "minqa"        
[169] "mitml"         "modeldata"     "ModelMetrics"  "modelr"       
[173] "modeltools"    "munsell"       "NbClust"       "nlme"         
[177] "nloptr"        "NMF"           "nnet"          "nnls"         
[181] "numDeriv"      "openssl"       "openxlsx"      "ordinal"      
[185] "pan"           "parallel"      "parallelly"    "parsnip"      
[189] "patchwork"     "pbdZMQ"        "pbkrtest"      "pillar"       
[193] "pkgbuild"      "pkgconfig"     "pkgload"       "pkgmaker"     
[197] "plogr"         "plotrix"       "plyr"          "png"          
[201] "polynom"       "prabclus"      "praise"        "prettyunits"  
[205] "pROC"          "processx"      "prodlim"       "progress"     
[209] "progressr"     "promises"      "proxy"         "ps"           
[213] "purrr"         "quadprog"      "quantmod"      "quantreg"     
[217] "R6"            "rappdirs"      "rcmdcheck"     "RColorBrewer" 
[221] "Rcpp"          "RcppArmadillo" "RcppEigen"     "readr"        
[225] "readxl"        "recipes"       "registry"      "rematch"      
[229] "remat